In [2]:
import pandas as pd
import numpy as np
import requests
from time import sleep

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import os
import json
import logging

In [4]:
# Driver creation
chrome_options = Options()
# chrome_options.add_argument("--headless") # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
# Silent download of drivers
logging.getLogger('WDM').setLevel(logging.NOTSET)
os.environ['WDM_LOG'] = 'False'
# Create service
webdriver_service = Service(ChromeDriverManager().install())
# Create driver
driver = webdriver.Chrome(service = webdriver_service, options = chrome_options)
# driver = webdriver.Chrome('F:\Thesis\Seller analysis\Selenium_scrape\chromedriver.exe')

In [5]:
# Opening goodreads login page using webdriver
driver.get("https://www.goodreads.com/ap/signin?language=en_US&openid.assoc_handle=amzn_goodreads_web_na&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.mode=checkid_setup&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.goodreads.com%2Fap-handler%2Fsign-in&siteState=0f9c274b6c7fab00499d20d3e6082b8d")

In [6]:
# Login into goodreads
password = "atharva5"
user = "athucool7@gmail.com"
userfield = driver.find_element(by = By.CSS_SELECTOR, value = "input[type = 'email']")
passfield = driver.find_element(by = By.CSS_SELECTOR, value = "input[type = 'password']")
userfield.send_keys(user)
passfield.send_keys(password)
driver.find_element(by = By.CSS_SELECTOR, value = "input[id = 'signInSubmit']").click()

In [7]:
def BookScraper(link):
    driver.get(link)
    try:
        driver.find_element(by = By.CSS_SELECTOR, value = "a#rating_details").text
        title = driver.find_element(by = By.CSS_SELECTOR, value = "h1#bookTitle").text
        rating = driver.find_element(by = By.CSS_SELECTOR, value = "span[itemprop = 'ratingValue']").text
        name = driver.find_element(by = By.CSS_SELECTOR, value = "a[class = 'authorName']").text
        num_ratings = driver.find_elements(by = By.CSS_SELECTOR, value  = "a[href = '#other_reviews']")[0].text[:-8]
        num_reviews = driver.find_elements(by = By.CSS_SELECTOR, value  = "a[href = '#other_reviews']")[1].text[:-8]
        num_followers = driver.find_element(by = By.CSS_SELECTOR, value  = "div.bookAuthorProfile__followerCount").text
        try:
            driver.find_element(by = By.CSS_SELECTOR, value = "a[onclick = 'swapContent($(this));; return false;']").click()
        except:
            synopsis =  driver.find_element(by = By.CSS_SELECTOR, value= "div[id = 'description']").text        
        else:
            synopsis =  driver.find_element(by = By.CSS_SELECTOR, value= "div[id = 'description']").text
    except:
        title = driver.find_element(by = By.CSS_SELECTOR, value = "h1[class = 'Text Text__title1']").text
        rating = driver.find_element(by = By.CSS_SELECTOR, value = "div[class = 'RatingStatistics__rating']").text
        name = driver.find_element(by = By.CSS_SELECTOR, value = "span[class = 'ContributorLink__name']").text
        num_ratings = driver.find_element(by = By.CSS_SELECTOR, value= "span[data-testid = 'ratingsCount']").text[:-8]
        num_reviews = driver.find_element(by = By.CSS_SELECTOR, value= "span[data-testid = 'reviewsCount']").text[:-8]
        author_profile = driver.find_element(by = By.CSS_SELECTOR, value = "div.AuthorPreview")
        num_followers = author_profile.find_element(by = By.CSS_SELECTOR, value = "span.u-dot-before").text[:-10]
        try:
            driver.find_element(by = By.CSS_SELECTOR, value = "button[class = 'Button Button--inline Button--small']").click()
        except:
            synopsis = driver.find_element(by = By.CSS_SELECTOR, value = "div[data-testid = 'description']").text    
        else:
            synopsis = driver.find_element(by = By.CSS_SELECTOR, value = "div[data-testid = 'description']").text
    return [title, rating, name, num_ratings, num_reviews, num_followers, synopsis]
    

In [22]:
df = pd.DataFrame(columns=['title', 'rating', 'name', 'num_ratings', 'num_reviews','num_followers', 'synopsis'])

In [23]:
from tqdm import tqdm_notebook

In [32]:
#for i in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
genre = 'fantasy'
for i in tqdm_notebook(range(9, 10), total = 2, unit = 'page'):    
    genrelink = "https://www.goodreads.com/shelf/show/" + genre + f"?page={i}"
    try:
        driver.get(genrelink)
        books = driver.find_elements(by = By.CSS_SELECTOR, value = "a.bookTitle")
    except:
        continue
    data = []
    book_links = []
    for j in range(len(books)):
        book_link = books[j].get_attribute('href')
        book_links.append(book_link)
    for k in tqdm_notebook(range(len(book_links)), total = len(book_links), unit = "book"):
        out = BookScraper(book_links[k])    
        df = df.append({'title' :out[0], 'rating' :out[1], 'name' :out[2], 'num_ratings' :out[3], 'num_reviews' :out[4],'num_followers' :out[5], 'synopsis' :out[6]}, ignore_index=True)

C:\Users\admin\AppData\Local\Temp/ipykernel_15440/1637930395.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(9, 10), total = 2, unit = 'page'):


  0%|          | 0/2 [00:00<?, ?page/s]

C:\Users\admin\AppData\Local\Temp/ipykernel_15440/1637930395.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for k in tqdm_notebook(range(len(book_links)), total = len(book_links), unit = "book"):


  0%|          | 0/50 [00:00<?, ?book/s]

In [38]:
df.drop_duplicates(subset = 'title', keep = 'first', inplace= True, ignore_index=True)
len(df)

99

In [39]:
df1 = pd.read_csv('books_fantasy.csv')
df1.drop_duplicates(subset = 'title', keep = 'first', inplace= True, ignore_index=True)
len(df1)

376

In [40]:
df1 = df1.append(df, ignore_index= True)
df1.drop_duplicates(subset = 'title', keep = 'first', inplace= True, ignore_index=True)
len(df1)

376

In [41]:
df1.to_csv("books_fantasy.csv")

PermissionError: [Errno 13] Permission denied: 'books_fantasy.csv'